In [1]:
import spacy
import cupy as cp
import timeit
nlp = spacy.load("nl_core_news_lg")
spacy.prefer_gpu()

True

In [2]:
def cosine_similarity_cupy(X, Y=None):
    if Y is None:
        Y = X
    
    # Ensure X and Y are CuPy arrays
    X = cp.asarray(X)
    Y = cp.asarray(Y)

    # Compute dot product
    dot_product = cp.dot(X, Y.T)

    # Compute norms
    norm_X = cp.linalg.norm(X, axis=1, keepdims=True)
    norm_Y = cp.linalg.norm(Y, axis=1, keepdims=True)

    # Compute cosine similarity
    cosine_sim = dot_product / (norm_X * norm_Y.T)

    return cosine_sim

In [3]:
def group_consecutive_similar_sentences(text, similarity_threshold=0.75):
    doc = nlp(text)
    
    valid_sentences = []
    valid_embeddings = []
    
    # Extract embeddings for valid sentences
    for sent in doc.sents:
        embedding = cp.array(sent.vector)  # Convert to CuPy array
        if not cp.all(embedding == 0):  # Check if the embedding is not zero
            valid_sentences.append(sent.text.strip())
            valid_embeddings.append(embedding)
    
    # Convert valid embeddings to a CuPy array
    valid_embeddings = cp.stack(valid_embeddings)
    
    if valid_embeddings.shape[0] == 0:
        return ["No valid sentences with embeddings found."]
    
    # Calculate cosine similarities using the CuPy implementation
    similarities = cosine_similarity_cupy(valid_embeddings)  # Full cosine similarity matrix
    
    grouped_sentences = []
    current_group = [valid_sentences[0]]
    
    for i in range(1, len(valid_sentences)):
        # Compare the current sentence with the previous one
        similarity = similarities[i, i - 1].item()  # Convert from CuPy to native Python float
        if similarity >= similarity_threshold:
            current_group.append(valid_sentences[i])
        else:
            grouped_sentences.append(current_group)
            current_group = [valid_sentences[i]]
    
    # Append the last group
    grouped_sentences.append(current_group)
    
    return grouped_sentences


In [4]:
text = """Vanaf 28 september 2024 neemt Antwerpen de fakkel van het Ensorjaar over van Oostende met een veelzijdig en verrassend expoprogramma. Wat Antwerpen heeft met Ensor? Een gedeelde, verrassende blik voorbij het alledaagse. Die gaat al terug tot de tijd van Ensor zelf. Niet toevallig kwamen veel van zijn werken nog tijdens zijn leven in de Scheldestad terecht. Ze vormen vandaag de kern van de Ensor-collectie van het KMSKA en een vertrekpunt voor het Ensor Research Project. In zijn oeuvre laat Ensor zich - net als Antwerpen - kennen als een game-changer: vaak met een knipoog, soms dwars en altijd innovatief. Eigenschappen die Ensor tijdloos en relevant maken. Antwerpen kiest daarom voor verrassende invalshoeken om zijn werk te belichten. Hoe zien we echo’s van Ensor in de kunst, mode en fotografie? Hoe blijft hij inspireren en wat kunnen we vandaag nog van hem leren?"""

In [5]:
def measure_grouping():
    grouped_sentences = group_consecutive_similar_sentences(text)

In [6]:
elapsed_time = timeit.timeit(measure_grouping, number=1000)
average_time = elapsed_time / 1000.0
print(f"Average time taken to group similar sentences: {average_time:.6f} seconds")
print(f"Elapsed time taken to group similar sentences 1000 times: {elapsed_time:.6f} seconds")

Average time taken to group similar sentences: 0.011605 seconds
Elapsed time taken to group similar sentences 1000 times: 11.605359 seconds


In [7]:
grouped_sentences = group_consecutive_similar_sentences(text)

for i, group in enumerate(grouped_sentences, 1):
    print(f"Groep {i}:")
    for sentence in group:
        print(f" - {sentence}")

Groep 1:
 - Vanaf 28 september 2024 neemt Antwerpen de fakkel van het Ensorjaar over van Oostende met een veelzijdig en verrassend expoprogramma.
Groep 2:
 - Wat Antwerpen heeft met Ensor?
Groep 3:
 - Een gedeelde, verrassende blik voorbij het alledaagse.
Groep 4:
 - Die gaat al terug tot de tijd van Ensor zelf.
 - Niet toevallig kwamen veel van zijn werken nog tijdens zijn leven in de Scheldestad terecht.
 - Ze vormen vandaag de kern van de Ensor-collectie van het KMSKA en een vertrekpunt voor het Ensor Research Project.
Groep 5:
 - In zijn oeuvre laat Ensor zich - net als Antwerpen - kennen als een game-changer: vaak met een knipoog, soms dwars en altijd innovatief.
 - Eigenschappen die Ensor tijdloos en relevant maken.
 - Antwerpen kiest daarom voor verrassende invalshoeken om zijn werk te belichten.
Groep 6:
 - Hoe zien we echo’s van Ensor in de kunst, mode en fotografie?
 - Hoe blijft hij inspireren en wat kunnen we vandaag nog van hem leren?
